In [6]:
import rdflib
from rdflib.plugins.sparql import prepareQuery
from tabulate import tabulate

In [7]:
filename = "../data/13/ABox.ttl"

In [8]:
text1 = '''CQ_13.1
Return the items, the name of the location their creation took place at, and eventually the name of the more precise location, if it exists.
'''

query1 = '''
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX lrmoo: <http://iflastandards.info/ns/lrm/lrmoo/>


SELECT ?item (GROUP_CONCAT(?place_label; separator=", ") AS ?place_label_combined)
WHERE {
    ?creation_event crm:P7_took_place_at ?place ;
        lrmoo:R17_created ?exp .
    ?exp lrmoo:R4i_is_embodied_in ?mnf .
    ?mnf lrmoo:R7i_is_exemplified_by ?item .
    ?place crm:P1_is_identified_by ?placeapp .
    ?placeapp crm:P190_has_symbolic_content ?place_label .
}
GROUP BY ?item
'''

In [9]:
text2 = '''CQ_13.2
Return the items and their current locations.
'''

query2 = '''
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX lrmoo: <http://iflastandards.info/ns/lrm/lrmoo/>


SELECT ?item ?place_label
WHERE {
    ?creation_event lrmoo:R17_created ?exp .
    ?exp lrmoo:R4i_is_embodied_in ?mnf .
    ?mnf lrmoo:R7i_is_exemplified_by ?item .
    ?item crm:P53_has_former_or_current_location ?place .
    ?place crm:P1_is_identified_by ?app .
    ?app crm:P190_has_symbolic_content ?place_label .
}
'''

In [10]:
queries = [(text1, query1),
            (text2, query2),
           ]

g = rdflib.ConjunctiveGraph()
g.parse(filename, format="turtle", encoding="utf-8")

for query in queries:
    q = prepareQuery(query[1])
    results = g.query(q)
    print(query[0])
    table = []
    for row in results:
        table.append([row[var] for var in results.vars])
    print(tabulate(table, headers=results.vars, tablefmt="psql"))

CQ_13.1
Return the items, the name of the location their creation took place at, and eventually the name of the more precise location, if it exists.

+----------------------------------------------------+------------------------+
| item                                               | place_label_combined   |
|----------------------------------------------------+------------------------|
| https://w3id.org/changes/4/cappellini/699693/itm/1 | Italia, Opicina        |
| https://w3id.org/changes/4/cappellini/699708/itm/1 | Italia, Spoleto        |
+----------------------------------------------------+------------------------+
CQ_13.2
Return the items and their current locations.

+----------------------------------------------------+---------------+
| item                                               | place_label   |
|----------------------------------------------------+---------------|
| https://w3id.org/changes/4/cappellini/699693/itm/1 | Stanza VIII   |
| https://w3id.org/changes/4/ca